# Basic AR Prey Analysis

In [ ]:
%matplotlib notebook
import os
import sys
sys.path.insert(0, r'..\..')
import paths

import seaborn as sb
import panel as pn
import holoviews as hv
from holoviews import opts, dim
hv.extension('bokeh')
from bokeh.resources import INLINE

import functions_bondjango as bd
from src import functions_kinematic as fk
import functions_plotting as fp
import functions_preprocessing as fpr
import functions_misc as fm
import functions_data_handling as fd
import pandas as pd
import numpy as np
import h5py

from scipy.stats import sem
import sklearn.decomposition as decomp
import umap
import sklearn.mixture as mix
from scipy.stats import sem

line_width = 5

In [ ]:
# Define a data loading function

def load_dataset(search_string, label=None, exclusion=None):
    # load the data
    # get the data paths
    try:
        data_path = snakemake.input[0]
    except NameError:
        # query the database for data to plot
        data_all = bd.query_database('analyzed_data', search_string)

        if exclusion is not None:
            for ds in data_all:
                if exclusion not in ds['analysis_path']:
                    data_path = ds['analysis_path']
                    data_date = ds['date']
                    break
        else:
            print(data_all[0].keys())
            data_path = data_all[0]['analysis_path']
            data_date = data_all[0]['date']
    print(data_path)
    print(data_date)

    # assemble a label for this data set
    if label is None:
        d = fd.parse_search_string(search_string)
        label = '_'.join([d['rig'], d['lighting'], d['result'], d['notes']])
    print('data label: ' + label + '\n')

    # load the data
    return pd.HDFStore(data_path), label, data_all[0]

In [ ]:
# define the name to be used for the saved figures
save_name = 'ARPrey_cricketcamo'

In [ ]:
### Load new data
# create container for holding multiple data sets
data_dict = {}

# Load real prey capture in the light - this is a baseline comparison
search_string = 'result:succ, lighting:normal, rig:ARPrey, analysis_type:preprocessing, notes:2D'
ds, label, DD = load_dataset(search_string, label="VScreen_2D")
# data_dict[label] = ds

# Get rid of doubled data set
# del ds

In [ ]:
summary_file = pd.ExcelFile(r'C:\Users\mmccann\Dropbox\My PC (P1-485)\Documents\ARPrey_results.xlsx', engine='openpyxl')
sheets = list(summary_file.sheet_names)

cricket_camo_df = pd.read_excel(summary_file, header=0, sheet_name='cricket_camo', dtype=object, engine='openpyxl')
moss_camo_df = pd.read_excel(summary_file, header=0, sheet_name='moss_camo', dtype=object, engine='openpyxl')
vr_cricket_camo_df = pd.read_excel(summary_file, header=0, sheet_name='VR_cricket_camo', dtype=object, engine='openpyxl')
cricket_camo_empty_spotlight_df = pd.read_excel(summary_file, header=0, sheet_name='cricket_camo_empty_spotlight', dtype=object, engine='openpyxl')
cricket_camo_dark_spot_df = pd.read_excel(summary_file, header=0, sheet_name='cricket_camo_dark_spot', dtype=object, engine='openpyxl')

In [ ]:
cc = cricket_camo_df.groupby(['spotlight', 'camo_albedo'])['capture_light', 'capture_dark'].sum().stack()
cc.index.names = ['spotlight', 'camo_albedo', 'outcome']
cc = cc.reset_index()
cc.columns.values[-1] = 'count'

In [ ]:
fg = sb.factorplot(x='camo_albedo', y='count', hue='outcome', 
                        col='spotlight', data=cc, kind='bar')

In [ ]:
mc = cricket_camo_df.groupby(['spotlight', 'camo_albedo'])['capture_light', 'capture_dark'].sum().stack()
mc.index.names = ['spotlight', 'camo_albedo', 'outcome']
mc = mc.reset_index()
mc.columns.values[-1] = 'count'
mc

In [ ]:
fg = sb.factorplot(x='camo_albedo', y='count', hue='outcome', 
                        col='spotlight', data=mc, kind='bar')

In [ ]:
vrcc = vr_cricket_camo_df.groupby(['spotlight', 'camo_albedo'])['interact_VR', 'ignore_VR'].sum().stack()
vrcc.index.names = ['spotlight', 'camo_albedo', 'outcome']
vrcc = vrcc.reset_index()
vrcc.columns.values[-1] = 'count'
vrcc

In [ ]:
fg = sb.factorplot(x='camo_albedo', y='count', hue='outcome', 
                        col='spotlight', data=vrcc, kind='bar')

In [ ]:
cces = cricket_camo_empty_spotlight_df.groupby(['spotlight', 'camo_albedo'])['interact_VR', 'ignore_VR'].sum().stack()
cces.index.names = ['spotlight', 'camo_albedo', 'outcome']
cces = cces.reset_index()
cces.columns.values[-1] = 'count'
cces

In [ ]:
fg = sb.factorplot(x='camo_albedo', y='count', hue='outcome', 
                        col='spotlight', data=cces, kind='bar')

In [ ]:
cricket_camo_dark_spot_df

In [ ]:
ccds = cricket_camo_dark_spot_df.groupby(['camo_albedo'])['capture_light', 'capture_dark', 'go_light', 'go_dark'].sum().stack()
ccds.index.names = ['camo_albedo', 'outcome']
ccds = ccds.reset_index()
ccds.columns.values[-1] = 'count'
ccds

In [ ]:
fg = sb.factorplot(x='camo_albedo', y='count', hue='outcome', 
                        data=ccds, kind='bar')